Importing required libraries

In [ ]:
import pandas as pd
import json
import numpy as np
import spacy
from spacy.lang.en import English
import string
import re

In [ ]:
import json

In [ ]:
!pip install datasets==1.0.2
!pip install transformers
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
rm: cannot remove 'seq2seq_trainer.py': No such file or directory
--2022-09-14 11:50:36--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-09-14 11:50:37 ERROR 404: Not Found.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset

In [ ]:
from transformers import EncoderDecoderModel
from transformers import  Seq2SeqTrainer,Seq2SeqTrainingArguments
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

Connecting to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
s_val_path = '/content/drive/MyDrive/corpus (1)/val.json'
s_test_path = '/content/drive/MyDrive/corpus (1)/test.json'
s_train_path = '/content/drive/MyDrive/corpus (1)/train.json'

Function to read dataset

In [ ]:
def read_samsum_dataset(path):
  with open(path) as in_file:
	  data = json.load(in_file)
  return data

In [ ]:
s_val = read_samsum_dataset(s_val_path)

In [ ]:
s_test = read_samsum_dataset(s_test_path)

In [ ]:
s_train = read_samsum_dataset(s_train_path)

In [ ]:
s_train_data = pd.DataFrame(s_train)
s_val_data = pd.DataFrame(s_val)
s_test_data = pd.DataFrame(s_test)

In [ ]:
s_test_data.head()

,id,summary,dialogue
0,13862856,Hannah needs Betty's number but Amanda doesn't...,"Hannah: Hey, do you have Betty's number?\nAman..."
1,13729565,Eric and Rob are going to watch a stand-up on ...,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...
2,13680171,Lenny can't decide which trousers to buy. Bob ...,"Lenny: Babe, can you help me with something?\r..."
3,13729438,Emma will be home soon and she will let Will k...,"Will: hey babe, what do you want for dinner to..."
4,13828600,Jane is in Warsaw. Ollie and Jane has a party....,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju..."


Removing \n, \r characters from train and val data

In [ ]:
s_train_data['dialogue'] = s_train_data['dialogue'].str.replace('\r', '')
s_train_data['dialogue'] = s_train_data['dialogue'].str.replace('\n', '')
s_train_data['summary'] = s_train_data['summary'].str.replace('\r', '')
s_train_data['summary'] = s_train_data['summary'].str.replace('\n', '')

In [ ]:
s_val_data['dialogue'] = s_val_data['dialogue'].str.replace('\r', '')
s_val_data['dialogue'] = s_val_data['dialogue'].str.replace('\n', '')
s_val_data['summary'] = s_val_data['summary'].str.replace('\r', '')
s_val_data['summary'] = s_val_data['summary'].str.replace('\n', '')

Removing stopwords

In [ ]:
sw1 =["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh","Hi" "hi", "hi", "Hi,","hid", "him", "'Hi,","himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy","hi" "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks","thanks,", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]

In [ ]:
def remove_stopwords(text): #remove stopwords
  words = [word for word in text.split() if word.lower() not in sw1]
  new_text = " ".join(words)
  return new_text

In [ ]:
s_val_data['dialogue'] = s_val_data.apply(lambda x: remove_stopwords(x.dialogue), axis=1)
s_val_data['summary'] = s_val_data.apply(lambda x: remove_stopwords(x.summary), axis=1)
s_train_data['dialogue'] = s_train_data.apply(lambda x: remove_stopwords(x.dialogue), axis=1)
s_train_data['summary'] = s_train_data.apply(lambda x: remove_stopwords(x.summary), axis=1)

In [ ]:
s_train_data.head()

,id,summary,dialogue
0,13818513,Amanda baked cookies bring Jerry tomorrow.,Amanda: I baked cookies. some?Jerry: Sure!Aman...
1,13728867,Olivia Olivier voting liberals election.,Olivia: voting election? Oliver: Liberals alwa...
2,13681000,Kim pomodoro technique recommended Tim stuff d...,"Tim: Hi, up?Kim: Bad mood tbh, I lots stuff en..."
3,13730747,Edward thinks love Bella. Rachel Edward open d...,"Edward: Rachel, I ove Bella..rachel: Dont else..."
4,13728094,"Sam confused, overheard Rick complaining roomm...",Sam: hey overheard rick somethingSam: i :-/Nao...


In [ ]:
s_train_data = s_train_data[:1000]

In [ ]:
s_val_data = s_val_data[:500]

Reading CNN dataset

In [ ]:
c_val_path = '/content/drive/MyDrive/cnn_dailymail/validation.csv'
c_test_path = '/content/drive/MyDrive/cnn_dailymail/test.csv'
c_train_path = '/content/drive/MyDrive/cnn_dailymail/train.csv'

In [ ]:
import pandas as pd

In [ ]:
def read_cnn_dataset(path):
  with open(path) as in_file:
	  data = pd.read_csv(in_file)
  return data

In [ ]:
c_val= read_cnn_dataset(c_val_path)

In [ ]:
c_test= read_cnn_dataset(c_test_path)

In [ ]:
c_train= read_cnn_dataset(c_train_path)

In [ ]:
c_train_data = pd.DataFrame(c_train)
c_val_data = pd.DataFrame(c_val)
c_test_data = pd.DataFrame(c_test)

In [ ]:
c_test_data = pd.DataFrame(c_test)
c_test_data.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
c_train_data.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
c_train_data['article'] = c_train_data['article'].str.replace('\r', '')
c_train_data['article'] = c_train_data['article'].str.replace('\n', '')
c_train_data['highlights'] = c_train_data['highlights'].str.replace('\r', '')
c_train_data['highlights'] = c_train_data['highlights'].str.replace('\n', '')

In [ ]:
c_val_data['article'] = c_val_data['article'].str.replace('\r', '')
c_val_data['article'] = c_val_data['article'].str.replace('\n', '')
c_val_data['highlights'] = c_val_data['highlights'].str.replace('\r', '')
c_val_data['highlights'] = c_val_data['highlights'].str.replace('\n', '')

In [ ]:
c_val_data.head()

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
c_train_data = c_train_data[:1000]

In [ ]:
c_val_data = c_val_data[:500]

In [ ]:
c_train_data.rename(columns={'article': 'text','highlights':'summary'}, inplace= True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
c_train_data.head(100)

,id,text,summary
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
95,004e6e935ea530b0992a89fd1307f7f41f4a234d,(CNN) -- If kings and queens are able to savor...,"Netherlands, Belgium and Qatar all had monarch..."
96,004f0f8c694c4b546b29565a8993a555537ff561,"PUBLISHED: . 07:04 EST, 9 January 2014 . | . U...","Zhu Sanni, 23, had been left alone at home for..."
97,004fc12e7cd2505a013d96e816afae3f3ce5015d,"Kabul, Afghanistan (CNN) -- Thousands of bottl...",Official: Bottles are almost exclusively from ...
98,00504275ede73591d94a6c1f994fd4856610421c,Even death couldn't part two skeletons excavat...,"Two skeletons were found holding hands, buried..."


Renaming the columns

In [ ]:
c_val_data.rename(columns={'article': 'text','highlights':'summary'}, inplace= True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
c_val_data.head()

,id,text,summary
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
c_test_data.rename(columns={'article': 'text','highlights':'summary'}, inplace= True)

In [ ]:
c_test_data.head()

,id,text,summary
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
s_train_data.rename(columns={'dialouge': 'text'}, inplace= True)

In [ ]:
s_train_data = s_train_data[['id','dialogue','summary']]
s_train_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. some?Jerry: Sure!Aman...,Amanda baked cookies bring Jerry tomorrow.
1,13728867,Olivia: voting election? Oliver: Liberals alwa...,Olivia Olivier voting liberals election.
2,13681000,"Tim: Hi, up?Kim: Bad mood tbh, I lots stuff en...",Kim pomodoro technique recommended Tim stuff d...
3,13730747,"Edward: Rachel, I ove Bella..rachel: Dont else...",Edward thinks love Bella. Rachel Edward open d...
4,13728094,Sam: hey overheard rick somethingSam: i :-/Nao...,"Sam confused, overheard Rick complaining roomm..."


In [ ]:
s_train_data.rename(columns={'dialogue': 'text','summary':'summary'}, inplace= True)

In [ ]:
s_train_data.head()

,id,text,summary
0,13818513,Amanda: I baked cookies. some?Jerry: Sure!Aman...,Amanda baked cookies bring Jerry tomorrow.
1,13728867,Olivia: voting election? Oliver: Liberals alwa...,Olivia Olivier voting liberals election.
2,13681000,"Tim: Hi, up?Kim: Bad mood tbh, I lots stuff en...",Kim pomodoro technique recommended Tim stuff d...
3,13730747,"Edward: Rachel, I ove Bella..rachel: Dont else...",Edward thinks love Bella. Rachel Edward open d...
4,13728094,Sam: hey overheard rick somethingSam: i :-/Nao...,"Sam confused, overheard Rick complaining roomm..."


In [ ]:
s_val_data = s_val_data[['id','dialogue','summary']]
s_val_data.head()

,id,dialogue,summary
0,13817023,"A: Tom, busy tomorrow’s afternoon?B: I’m prett...",animal shelter tomorrow puppy son. visited she...
1,13716628,Emma: I’ve fallen love advent calendar! Awesom...,Emma Rob love advent calendar. Lauren fits ins...
2,13829420,Jackie: Madison pregnantJackie: wanna talk itI...,Madison pregnant talk it. Patricia Stevens mar...
3,13819648,Marla: <file_photo>Marla: I bedKiki: lolTamara...,Marla pair boxers bed.
4,13728448,Robert: Hey address music shop mentioned befor...,Robert Fred send address music shop buy guitar...


In [ ]:
s_val_data.rename(columns={'dialogue': 'text','summary':'summary'}, inplace= True)

In [ ]:
s_test_data = s_test_data[['id','dialogue','summary']]
s_test_data.head()

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [ ]:
s_test_data.rename(columns={'dialogue': 'text','summary':'summary'}, inplace= True)

In [ ]:
s_test_data.head()

,id,text,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [ ]:
display(s_train_data.shape,c_train_data.shape)

(1000, 3)

(1000, 3)

In [ ]:
merged_train = pd.concat([s_train_data,c_train_data])

In [ ]:
from sklearn.utils import shuffle

In [ ]:
merged_train = shuffle(merged_train)

In [ ]:
merged_train.head()

,id,text,summary
449,015007887b47cb2267f7082c2ef90b58fe96589f,"Tick tock. Tick tock. In Roy Keane’s world, ev...",Republic of Ireland assistant Roy Keane spoke ...
501,017d27d00eb43678c15cb4a8dd4723a035323219,The taboo that sometimes still surrounds the s...,Fertility expert and author Emma Cannon has la...
54,13715772,Gabi: guys Christmas? $100.Jack: I it...Mary: ...,"Mary, Kate, Peter John items Christmas. Gabi s..."
841,13729738,Addie: Babes check makeup here?Maya: things le...,Maya send Addie list cosmetics picked online A...
205,00a39c134080b6f215a81c15d46c3ac7cc7bdcf3,By . Lucy Crossley . Tucked away under a grass...,"Developers had to excavate 5,000 tons of soil ..."


In [ ]:
merged_val = pd.concat([s_val_data,c_val_data])

In [ ]:
merged_val = shuffle(merged_val)

In [ ]:
merged_test = pd.concat([s_test_data,c_test_data])

In [ ]:
merged_test = shuffle(merged_test)

In [ ]:
merged_test.head()

,id,text,summary
4177,92151f31c3478c5e0e4352a0f7323c3acfc297e7,A drug-addicted Ryanair steward who swiped a £...,Teacher Adam Galloway got home and found Nikon...
1486,d54fa2d25e71a61f2de578fe426e532dee6939a1,The parents of a 5-year-old boy whose body was...,"Ashley White, 30, and Paul Thomas, 32, charged..."
5747,70490801462b9429957244dc2bb66638a0e1029a,Clare Balding has admitted she was on TV too m...,Broadcaster vows to do less after hosting ever...
9863,a7760a436de7ff6000297b21a7b52890cc6d1ae0,Alex Hales claims he is still waiting for a 'p...,Alex Hales has nine England caps since his deb...
2801,c71065c50b7f67b964bea47623e31c93a9903fb0,"She might be celebrating her 75th birthday, bu...",Queen Margrethe II of Denmark is celebrating h...


In [ ]:
from datasets import Dataset
train_data=Dataset.from_pandas(merged_train)
val_data = Dataset.from_pandas(merged_val)
test_data = Dataset.from_pandas(merged_test)

In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
    

In [ ]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model_checkpoint = "t5-base"
metric = load_metric("rouge")

Tokenizing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Converting into model inputs

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
#batch_size=256

In [ ]:
train_tokenized_datasets = train_data.map(preprocess_function, batched=True)#,batch_size=batch_size)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
val_tokenized_datasets = val_data.map(preprocess_function, batched=True)#,batch_size=batch_size)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

Instantiating T5 model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

Defining model arguments for fine-tuning

In [ ]:
batch_size = 8

#output_dir = '/content/drive/MyDrive'
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_steps=1
    #fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Function for evaluate metrics using ROUGE

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Training the model

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, summary, text, id. If __index_level_0__, summary, text, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 625


Epoch,Training Loss,Validation Loss,Rouge1,Rougel,Gen Len
1,1.928800,1.950939,31.657400,27.182200,15.089000
2,1.834200,1.885769,32.999300,28.354500,14.796000
3,1.640500,1.869518,33.899700,28.793900,15.135000
4,2.027700,1.865224,34.009200,28.750500,15.005000
5,2.053800,1.862678,34.157000,28.903200,15.085000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, summary, text, id. If __index_level_0__, summary, text, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, summary, text, id. If __index_level_0__, summary, text, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, summary, text, id. If __index_level_0__, summary, text, id are not

TrainOutput(global_step=625, training_loss=1.988896845817566, metrics={'train_runtime': 1222.3756, 'train_samples_per_second': 8.181, 'train_steps_per_second': 0.511, 'total_flos': 6068331984199680.0, 'train_loss': 1.988896845817566, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, summary, text, id. If __index_level_0__, summary, text, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 1.8626779317855835,
 'eval_rouge1': 34.157,
 'eval_rougeL': 28.9032,
 'eval_gen_len': 15.085,
 'eval_runtime': 79.3697,
 'eval_samples_per_second': 12.599,
 'eval_steps_per_second': 1.575,
 'epoch': 5.0}

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Function to generate summaries on test data

In [ ]:
def summary_gen(input_text):
  with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=5,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

  return  tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
merged_test['pred']= merged_test[:10].apply(lambda x: summary_gen(x.text), axis=1)

In [ ]:
merged_test.head()

,id,text,summary,pred
4177,92151f31c3478c5e0e4352a0f7323c3acfc297e7,A drug-addicted Ryanair steward who swiped a £...,Teacher Adam Galloway got home and found Nikon...,"Aaron Galloway, 23, was on a half-term city br..."
1486,d54fa2d25e71a61f2de578fe426e532dee6939a1,The parents of a 5-year-old boy whose body was...,"Ashley White, 30, and Paul Thomas, 32, charged...","Noah Thomas, five, was last seen watching cart..."
5747,70490801462b9429957244dc2bb66638a0e1029a,Clare Balding has admitted she was on TV too m...,Broadcaster vows to do less after hosting ever...,Clare Balding admitted she was on TV too much ...
9863,a7760a436de7ff6000297b21a7b52890cc6d1ae0,Alex Hales claims he is still waiting for a 'p...,Alex Hales has nine England caps since his deb...,Big-hitting batsman has nine England caps to d...
2801,c71065c50b7f67b964bea47623e31c93a9903fb0,"She might be celebrating her 75th birthday, bu...",Queen Margrethe II of Denmark is celebrating h...,Queen Margrethe of Denmark welcomed royals fro...
